## Pilot for downsampling / throwing in errors
Plan: Input is a a HDF5, Output is a HDF5. Eventually make class that can downsample, and/or throw errors on the reads.

Class is a wrapper of an HDF5, and applies the operations to it.

In [1]:
import allel
import h5py  # Python Package to do the HDF5.
import numpy as np
import pandas as pd
import socket
import os

### Pick the right path (whether on cluster or at home)
if socket.gethostname() == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket.gethostname() == "midway2-0401.rcc.local" or socket.gethostname() == "midway2-0402.rcc.local":
    print("Midway jnovmbre partition detected.")
    path = "/project/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)
print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns

Midway jnovmbre partition detected.
/project/jnovembre/hringbauer/HAPSBURG


In [37]:
class ModifyHDF5Genotypes(object):
    """Class for Modifying HDF4 genotypes"""

    f = 0    # The hdf5 object to modify
    original_path = "" # Where to find the original HDF5
    save_path = ""  # Where to save the modified HDF5 to
    output = True # Whether to print any output
    gt_new = []

    def __init__(self, original_path="", save_path="", output=True):
        """pop_path: Where to load a HDF5 from
           save_path: Where to save the new HDF5 to"""
        self.output = output
        self.save_path = save_path
        
        if output == True:
            print("Heyho back old friend. I started running")
        
        if len(original_path)>0:
            self.original_path = original_path
            self.load_data()
        else:
            print("No HDF5 Loaded! Alarm. Alarm. Alarm.")

    def load_data(self, path=""):
        """Load the HDF5 Data"""
        if len(path)==0:
            path = self.original_path
        self.f = h5py.File(path, "r") # Load for Sanity Check. See below!
        
        if self.output == True:
            print("Loaded HDF5")
            print("Loaded %i variants" % np.shape(self.f["calldata/GT"])[0])
            print("Loaded %i individuals" % np.shape(self.f["calldata/GT"])[1])
            print(list(self.f["calldata"].keys()))
            print(list(self.f["variants"].keys()))
            #self.f["samples"] # Samples Vector
        
        ### Sanity Check whether both Genotypes are there and nothing else
        assert(np.min(self.f["calldata/GT"]) == 0)
        assert(np.max(self.f["calldata/GT"]) == 1)

    def save_data(self, gt, ad, ref, alt, pos, rec, samples, path):
        """Create a new HDF5 File with Input Data.
        gt: Genotype data [l,k,2]
        ad: Allele depth [l,k,2]
        ref: Reference Allele [l]
        alt: Alternate Allele [l]
        pos: Position  [l]
        m: Map position [l]
        samples: Sample IDs [k]"""

        l, k, _ = np.shape(gt)  # Nr loci and Nr of Individuals

        if os.path.exists(path):  # Do a Deletion of existing File there
            os.remove(path)

        dt = h5py.special_dtype(vlen=str)  # To have no problem with saving

        with h5py.File(path, 'w') as f0:
            # Create all the Groups
            f_map = f0.create_dataset("variants/MAP", (l,), dtype='f')
            f_ad = f0.create_dataset("calldata/AD", (l, k, 2), dtype='i')
            f_ref = f0.create_dataset("variants/REF", (l,), dtype=dt)
            f_alt = f0.create_dataset("variants/ALT", (l,), dtype=dt)
            f_pos = f0.create_dataset("variants/POS", (l,), dtype='i')
            f_gt = f0.create_dataset("calldata/GT", (l, k, 2), dtype='i')
            f_samples = f0.create_dataset("samples", (k,), dtype=dt)

            # Save the Data
            f_map[:] = rec
            f_ad[:] = ad
            f_ref[:] = ref.astype("S1")
            f_alt[:] = alt.astype("S1")
            f_pos[:] = pos
            f_gt[:] = gt
            f_samples[:] = np.array(samples).astype("S10")

        if self.output == True:
            print(f"Successfully saved {k} individuals to: {path}")

    def create_error_gt(self, freq_flips=0.01):
        """Create Error on the HDF5 of genotypes.
        freq_flips: How often to do flip of genotyps"""
        f = self.f
        gt = f["calldata/GT"]
        switch = np.random.random(np.shape(gt)) < freq_flips
        
        if self.output == True:
            print(f"Swapping frac of SNPs: {np.mean(switch):.6f}")

        ### Switch the Genotypes
        gt_new = (gt + switch) %2
        ### Debugging
        #print(np.mean(gt_new != gt))
        #print(np.unique(gt_new))
        
        self.save_data(gt_new, f["calldata/AD"], f["variants/REF"][:], f["variants/ALT"][:], f["variants/POS"], 
                       f["variants/MAP"], f["samples"][:], self.save_path)
            
    def downsample_gt(self, frac=0.9):
        """Downsample the HDF5 to fewer reads.
        Update also the recombination and position map if needed to remove missing values
        frac: To what fraction of markers one downsamples"""
        f = self.f
        gt = f["calldata/GT"]
        
        ### Decide on SNPs
        l, n, _ = np.shape(gt)
        survive = np.random.random(l) < frac
        print(f"Fraction Loci surviving {np.mean(survive):.6f}")
        
        ### Downsample
        gt_new = gt[survive,:,:]   #
        r_map_new = f["variants/MAP"][survive]
        ad_new = f["calldata/AD"][survive,:,:]
        ref_new = f["variants/REF"][survive]
        alt_new = f["variants/ALT"][survive]
        pos_new = f["variants/POS"][survive]
        
        ### Downsample where needed  
        self.save_data(gt_new, ad_new, ref_new, alt_new, pos_new, r_map_new, f["samples"], self.save_path)
        
    def copy_rohinfo(self, load_path="", save_path="", file="roh_info.csv"):
        """Copy in the ROH Info from folder of load path into folder of save_path.
        file: Which file to copy (roh_info by default)"""
        if len(load_path) == 0:
            load_path = self.original_path
            
        if len(save_path) ==0 :
            save_path = self.save_path
            
        save_path = os.path.dirname(save_path) + "/" + file
        load_path = os.path.dirname(load_path) + "/" + file
        
        ### Copy the file
        !cp $load_path $save_path   

## Prepare data hdf5s with various levels of errors

In [38]:
error_vec = np.logspace(-3,-1, 8)
org_folder = "./Simulated/1000G_Mosaic/TSI5/"
lengths = [0, 2, 4, 6, 8, 10]

for l in lengths:
    for e in error_vec:
        print(f"Doing Error Frac: {e}")
        org_folder1 = org_folder + "ch3_" + str(l) + "cm/"
        load_path = org_folder1 + "data.h5" 
        
        e_print = str(round(e, 4)).split(".")[1] # Extract four digits after decimal
        
        ### To do save 4 digits error data file
        save_path = org_folder1 + "error/" + e_print + "/data.h5"   
        
        # Make Directory if not already there
        if not os.path.exists(os.path.dirname(save_path)):   
            os.makedirs(os.path.dirname(save_path))
        
        #os.remove(save_path)  # For previous whoopsie
        m = ModifyHDF5Genotypes(original_path=load_path, save_path=save_path)
            
        m.create_error_gt(freq_flips=e)
        m.copy_rohinfo()   # Copy the ROH Info!!

Doing Error Frac: 0.001
Heyho back old friend. I started running
Loaded HDF5
Loaded 77652 variants
Loaded 100 individuals
['AD', 'GT']
['ALT', 'MAP', 'POS', 'REF']
Doing Error Frac: 0.0019306977288832496
Heyho back old friend. I started running
Loaded HDF5
Loaded 77652 variants
Loaded 100 individuals
['AD', 'GT']
['ALT', 'MAP', 'POS', 'REF']
Doing Error Frac: 0.003727593720314938
Heyho back old friend. I started running
Loaded HDF5
Loaded 77652 variants
Loaded 100 individuals
['AD', 'GT']
['ALT', 'MAP', 'POS', 'REF']
Doing Error Frac: 0.0071968567300115215
Heyho back old friend. I started running
Loaded HDF5
Loaded 77652 variants
Loaded 100 individuals
['AD', 'GT']
['ALT', 'MAP', 'POS', 'REF']
Doing Error Frac: 0.013894954943731374
Heyho back old friend. I started running
Loaded HDF5
Loaded 77652 variants
Loaded 100 individuals
['AD', 'GT']
['ALT', 'MAP', 'POS', 'REF']
Doing Error Frac: 0.026826957952797246
Heyho back old friend. I started running
Loaded HDF5
Loaded 77652 variants
Load

### Prepare the indivdiuals with various levels of missingness 

### Area 51

In [36]:
m = ModifyHDF5Genotypes(original_path="./Simulated/1000G_Mosaic/TSI5/ch3_2cm/data.h5", save_path="./Simulated/1000G_Mosaic/TSI5/ch3_2cm/data_shitty.h5")
#m.downsample_gt(frac=0.8)
#m.create_error_gt(freq_flips=0.01)
m.copy_rohinfo()

Heyho back old friend. I started running
Loaded HDF5
Loaded 77652 variants
Loaded 100 individuals
['AD', 'GT']
['ALT', 'MAP', 'POS', 'REF']
./Simulated/1000G_Mosaic/TSI5/ch3_2cm/roh_info.csv
./Simulated/1000G_Mosaic/TSI5/ch3_2cm/roh_info.csv
